In [1]:
# Core
import numpy as np
import pandas as pd

In [2]:
# Local imports
import kepler_sieve
from asteroid_direction import ast, c, mpd, dpm, prep_ast_block, calc_dir_ast_block
from asteroid_spline import make_spline_ast_dir
from sky_patch import dir2SkyPatchID
from db_utils import sp2df, df2db

# Build block of asteroid directions at high resolution

In [3]:
# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
# interval_min: int = 1
interval_min: int = 5

# Asteroid block
n0: int = 1
n1: int = 11

In [4]:
# Prepare asteroid block
t_obs, asteroid_id = prep_ast_block(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, interval_min=interval_min)

In [5]:
# Build spline of asteroid direction
spline_u = make_spline_ast_dir(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [6]:
# Calculate asteroid directions from position and velocity
u_ast, light_time = spline_u(t_obs, asteroid_id)

In [7]:
# The TimeID
time_id = np.round(t_obs * mpd).astype(np.int64)

In [8]:
# Calculate the SkyPatchID
N_sp = 1024
sky_patch_id = dir2SkyPatchID(dir=u_ast, N=N_sp)

In [9]:
# Calculate the asteroid number in this batch from asteroid_id
asteroid_id_unq = np.unique(asteroid_id)
N_ast = asteroid_id_unq.shape[0]
ast_bn_unq = np.arange(N_ast, dtype=np.int)
ast_bn = np.searchsorted(asteroid_id_unq, asteroid_id)

In [10]:
# ast_bn

In [12]:
# Number of rows
N_row = asteroid_id.shape[0]

In [38]:
# Rows with a new asteroid
is_new_ast = np.ones(N_row, dtype=np.bool)
is_new_ast[1:N_row] = (asteroid_id[1:N_row] != asteroid_id[0:N_row-1])
idx_new_ast = np.argwhere(is_new_ast).flatten()

In [ ]:
# Rows with a new sky patch entry; includes row with a new asteroid
is_new_skypatch = np.ones(N_row, dtype=np.bool)
is_new_skypatch[1:N_row] = (sky_patch_id[1:N_row] != sky_patch_id[0:N_row-1])

In [29]:
# Flag for last segment
is_last_segment = np.zeros(N_row, dtype=np.bool)
is_last_segment[idx_new_ast-1] = True

In [39]:
# Combined mask for interior segments
mask = (is_new_ast | is_new_skypatch) & (~is_last_segment)

In [40]:
# Row numbers for these masks
idx = np.argwhere(mask).flatten()

In [41]:
# The segment number, increasing across all asteroids in batch
sn_cum = np.cumsum(mask)
sn_base = sn_cum[is_new_ast]
sn = sn_cum - sn_base[ast_bn]

In [47]:
# Wrap into DataFrame
df_tbl = {
    'AsteroidID': asteroid_id,
    'Segment': sn,
    'TimeID': time_id,
    'tObs': t_obs,
    'ux': u_ast[:, 0],
    'uy': u_ast[:, 1],
    'uz': u_ast[:, 2],
    'LightTime': light_time,
    'SkyPatchID': sky_patch_id,
}

df_all = pd.DataFrame(df_tbl)

In [48]:
df_all

,AsteroidID,Segment,TimeID,tObs,ux,uy,uz,LightTime,SkyPatchID
0,1,0,83520000,58000.000000,-0.378601,0.924673,0.040506,26.537850,6382172
1,1,0,83520005,58000.003472,-0.378623,0.924664,0.040508,26.537543,6382172
2,1,0,83520010,58000.006944,-0.378644,0.924655,0.040511,26.537237,6382172
3,1,0,83520015,58000.010417,-0.378665,0.924647,0.040513,26.536930,6382172
4,1,0,83520020,58000.013889,-0.378687,0.924638,0.040516,26.536623,6382172
...,...,...,...,...,...,...,...,...,...
5760005,10,11744,86399980,59999.986111,0.452085,-0.891974,0.001213,29.877185,18877959
5760006,10,11744,86399985,59999.989583,0.452105,-0.891964,0.001213,29.876963,18877959
5760007,10,11744,86399990,59999.993056,0.452126,-0.891953,0.001214,29.876741,18877959
5760008,10,11744,86399995,59999.996528,0.452146,-0.891943,0.001215,29.876518,18877959


In [44]:
# TimeID of each segment
time_id_0 = time_id[idx]
time_id_1 = time_id[idx+1]

In [51]:
# Create DataFrame for just the rows that start interior segments
cols = ['AsteroidID', 'Segment', 'SkyPatchID']
df = df_all.loc[mask, cols]

In [53]:
df['TimeID_0'] = time_id_0
df['TimeID_1'] = time_id_1

In [54]:
df

,AsteroidID,Segment,SkyPatchID,TimeID_0,TimeID_1
0,1,0,6382172,83520000,83520005
27,1,1,6382171,83520135,83520140
45,1,2,6384219,83520225,83520230
63,1,3,6384218,83520315,83520320
99,1,4,6384217,83520495,83520500
...,...,...,...,...,...
5759870,10,11740,18877955,86399305,86399310
5759904,10,11741,18877956,86399475,86399480
5759938,10,11742,18877957,86399645,86399650
5759972,10,11743,18877958,86399815,86399820


In [55]:
dt = (df.TimeID_1 - df.TimeID_0).values

In [57]:
np.min(dt)

5